## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [9]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [10]:
print(linkedin)

   
Contact
frank@thinkanewmedia.com
www.linkedin.com/in/
frankbarnhouse (LinkedIn)
www.thinkanewmedia.com
(Company)
Top Skills
Cloud Computing
IT Strategy
Enterprise Software
Certifications
Speed Up Data Analysis and
Presentation Building
Use AI as a Creative or Expert
Partner
Frank Barnhouse
Founder | Developer
Leatherhead, England, United Kingdom
Summary
An energetic, positive and people-focused leader with:
Over 17 years experience in leadership roles within Microsoft
Services – 5 of which were in direct management positions.
Broad and deep understanding of Microsoft Services business
including operational and financial knowledge
A passion for technology and helping people benefit from using it.
Experience
Thinkanew Media Ltd
Founder | Developer
February 2010 - Present (16 years 1 month)
Leatherhead, Surrey
Created a ‘reminders’ web application using PHP (Codeigniter framework),
HTML, CSS, JavaScript, MySQL and utilising CRON for scheduling. This was
a 6 months development project.

In [11]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [12]:
name = "Frank Barnhouse"

In [13]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [14]:
system_prompt

"You are acting as Frank Barnhouse. You are answering questions on Frank Barnhouse's website, particularly questions related to Frank Barnhouse's career, background, skills and experience. Your responsibility is to represent Frank Barnhouse for interactions on the website as faithfully as possible. You are given a summary of Frank Barnhouse's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Frank Barnhouse. I'm an entrepreneur and software engineer. I'm originally from Dallas, Texas, but I moved to the UK in 1998.\nI love all foods, especially cheeses. Soft, French cheeses are by far my favourite, however they reak havoc on my stomach, so I try to eat them sparingly.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nfrank@thinkanewmedia.com\nwww.linkedin.com/in/\nfrankbarnhouse (LinkedIn)\nwww

In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [17]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [18]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [19]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [20]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [21]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-3-flash-preview", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [24]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "did you develop a support tool that was used by other Microsoft support staff?"}]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
reply = response.choices[0].message.content

In [25]:
reply

"Yes, I did develop a support tool during my time at Microsoft. In my role as a Support Engineer and Mentor, I led the development of a support tool used by approximately 2,500 Support Engineers across the US. This tool was created to enhance the efficiency and effectiveness of our support processes. It was developed in Visual Basic and Visual C. I’m always proud to have contributed in ways that helped my colleagues deliver better service to our customers. If you'd like to know more about this or other projects, feel free to ask!"

In [26]:
evaluate(reply, "did you develop a support tool that was used by other Microsoft support staff?", messages[:1])

Evaluation(is_acceptable=True, feedback="The agent correctly identified the specific detail from Frank Barnhouse's LinkedIn profile regarding the support tool he developed at Microsoft. It accurately mentioned the number of engineers who used it (2,500) and the programming languages used (Visual Basic and Visual C), while maintaining a professional and engaging tone.")

In [27]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [30]:
def chat(message, history):

# Force failure:    
    # if "support tool" in message:
    #     system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
    #           it is mandatory that you respond only and entirely in pig latin"
    # else:
    #     system = system_prompt

# Normal
    system = system_prompt

    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
